In [1]:
from ngsolve import *
import netgen.gui
import numpy as np
import scipy
import scipy.sparse as sparse
import math
from netgen.geom2d import *
import scipy.sparse as sp
import matplotlib.pyplot as plt

In [ ]:
geo = SplineGeometry()

r1 = 2.5
r2 = 1
h1 = 5
h2 = 1.5
h3 = 1
d = 0.2


x_val = [0, 2*r1, 2*r1, r1+r2, r1+r2, r1+r2-d, r1+r2-d, 2*r1-d, 2*r1-d, d, d, r1-r2+d, r1-r2+d, r1-r2, r1-r2, 0]
y_val = [0, 0, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1, d, d, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1]

assert len(x_val)==len(y_val), "lengths of x and y values must be the same"
points = [None]*len(x_val)
amount_points = len(points)
for j in range(0,amount_points):
    points[j] = geo.AppendPoint(x_val[j], y_val[j])

for j in range(0,amount_points):
    geo.Append(["line", points[j], points[(j+1) % amount_points]], leftdomain=0, rightdomain=1, bc = "bottle")

geo.AddRectangle((-1,-1),(2*r1+1, h1+h2+h3+1), bc="robin")
geo.SetMaterial (1, "air")
geo.SetMaterial (2, "glass")

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
Draw(mesh)

fes = H1(mesh, order=5, complex=True, definedon="air")#, dirichlet="bottle")

## peak on top of bottle
source1 = 1000*exp(-(4**2)*((x-2.5)**2 + (y-(h1+h2+h3+0.3))**2))

source2 = 100*exp(-(4**2)*((x-2.5)**2 + (y+0.5)**2))
Draw(source2,mesh,name = "pulse")

In [2]:
geo = SplineGeometry()

r1 = 2.5
r2 = 1
h1 = 5
h2 = 1.5
h3 = 1
d = 0.2


x_val = [0, 2*r1, 2*r1, r1+r2, r1+r2, r1+r2-d, r1+r2-d, 2*r1-d, 2*r1-d, d, d, r1-r2+d, r1-r2+d, r1-r2, r1-r2, 0]
y_val = [0, 0, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1, d, d, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1]

assert len(x_val)==len(y_val), "lengths of x and y values must be the same"
points = [None]*len(x_val)
amount_points = len(points)
for j in range(0,amount_points):
    points[j] = geo.AppendPoint(x_val[j], y_val[j])

for j in range(0,amount_points):
    geo.Append(["line", points[j], points[(j+1) % amount_points]], leftdomain=2, rightdomain=1, bc = "bottle")

geo.AddRectangle((-1,-1),(2*r1+1, h1+h2+h3+1), leftdomain=1, bc="robin")
geo.SetMaterial (1, "air")
geo.SetMaterial (2, "glass")

mesh = Mesh(geo.GenerateMesh(maxh=0.2))
Draw(mesh)

fes = H1(mesh, order=5, complex=True, definedon="air")#, dirichlet="bottle")

## peak on top of bottle
source1 = 1000*exp(-(4**2)*((x-2.5)**2 + (y-(h1+h2+h3+0.3))**2))

source2 = 100*exp(-(4**2)*((x-2.5)**2 + (y+0.5)**2))
Draw(source2,mesh,name = "pulse")

In [ ]:
maxind = 20
#fes = H1(mesh, order=5,complex = True)
fes = H1(mesh, order=5,dirichlet=".*")
gfu = GridFunction(fes,name = "u")
erg = np.zeros([len(gfu.vec),maxind],dtype = complex)
fh = np.zeros(len(gfu.vec),dtype = complex)

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

A = BilinearForm(fes)
K = BilinearForm(fes)
M = BilinearForm(fes)
F = LinearForm(fes)
#R = BilinearForm(fes)

omega = Parameter(1)
A += (grad(u)*grad(v)- omega**2*u*v)*dx
K += grad(u)*grad(v)*dx
M += u*v*dx
#R += -1j*u*v*ds(definedon=mesh.Materials('glas'))
#A += -1j*omega*u*v*ds(definedon=mesh.Materials('glas'))


#A += 1j*omega*u*v*ds # R 
F += source2*v*dx  
o = []  

norm = []

#o = verg[0:21]
for i in range(0,maxind):
    o.append(i)

j = 0
Draw(gfu)
for i in o:   
    omega.Set(i) 
    A.Assemble()
    K.Assemble()
    M.Assemble()
    F.Assemble()  
    #R.Assemble() 
    gfu.vec.data = A.mat.Inverse(fes.FreeDofs()) * F.vec
    erg[:,j] = gfu.vec
    fh = F.vec
    norm.append(sqrt (Integrate ((gfu)*(gfu),mesh)))
    print("Number of iter: ",j,"omega: ",i,"L2-Norm:",sqrt (Integrate ((gfu)*(gfu),mesh)))
    j = j+1
#As = A.mat
Ms = M.mat
Ks = K.mat
#Rs = R.mat
rows,cols,vals = Ms.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))
rows,cols,vals = Ks.COO()
Kh = sp.csr_matrix((vals,(rows,cols)))
#rows,cols,vals = Rs.COO()
#Rh = sp.csr_matrix((vals,(rows,cols)))
#rows,cols,vals = As.COO()
#Ah = sp.csr_matrix((vals,(rows,cols)))
#Ah = sp.csr_matrix.toarray(Ah)

In [ ]:
V, r = np.linalg.qr(erg)
V = erg.dot(np.linalg.inv(r))
print(np.vdot(V[:,1],(V[:,2])))
len(V)

In [ ]:
plt.plot(o[1:20],norm[1:20])
plt.xlabel("omega")
plt.ylabel("Euklidic-Norm")
#plt.savefig('results/resonanz_20.png')

In [ ]:
MN = np.transpose(V).dot(Mh.dot(V))
KN = np.transpose(V).dot(Kh.dot(V))
fN = np.transpose(V).dot(fh)
#RN = np.transpose(V).dot(Rh.dot(V))

In [ ]:
iteration = int(1000)
interval = 20
factor = interval / iteration
omegaN = []
norm_L2 =[]
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    omegaN.append(factor*i)
    #uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN+omegaN[i]*RN, fN)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, fN)
    new = uN[:,i].dot(np.transpose(V))
    gif = GridFunction(fes,name = "gif")
    gif.vec.FV().NumPy()[:] = new
    Draw(gif)
    print("i: ",i,"Omega: ",omegaN[i])
    #input()
    Redraw()
    #norm_L2.append(sqrt(uN[:,i].dot(uN[:,i])))

In [ ]:
iteration = int(1e4)
interval = 20
factor = interval / iteration
omegaN = []
norm_L2 =[]
res = []
uN = np.zeros([maxind,iteration])
for i in range(iteration):
    print(i)
    omegaN.append(factor*i)
    uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, fN)
    res.append(np.linalg.norm((Kh-omegaN[i]**2*Mh).dot(uN[:,i].dot(np.transpose(V)))-fN.dot(np.transpose(V))))
    norm_L2.append(sqrt(uN[:,i].dot(uN[:,i])))

In [ ]:
#plt.semilogy(verg[0:maxpoints],point[0:maxpoints],'*r')
plt.semilogy(omegaN[10:],norm_L2[10:],'b')
plt.xlabel("omega")
plt.ylabel("Euclidic Norm")
#plt.savefig('results/resonanz_RB_20.png')